In [1]:
from cranes import *

set_plot_formatting()

#### set the job name for the output file

In [2]:
job_name = 'resimulate_case_study'

#### specify the input files

In [3]:
input_data_folder = 'inputs/'
inventory_filename = input_data_folder + 'SF_Downtown_Inventory.csv'
ground_motion_filename = input_data_folder + 'SF_Mw7.2_SanAndreas_GroundMotionMaps.h5'
original_vulnerability_filename = input_data_folder + 'Vulnerabilities_original.h5'
retrofit_vulnerability_filename = input_data_folder + 'Vulnerabilities_retrofits.h5'

input_filenames = [inventory_filename, ground_motion_filename, original_vulnerability_filename, retrofit_vulnerability_filename]
output_filename = 'outputs/' + job_name + '_results.h5'

#### set up the analysis case organization

In [4]:
# optional test run with only one case

cases = {'baseline':           {'i_damage': 0,
                                'i_impeding_factors': 0,
                                'i_cordons': 0,
                                'description': 'Cordon Extent: 1.5H, REDi Impeding Factors'
                               },
        }

In [5]:
# set up damage, impeding factor, and cordon indices for storing each each
# index number is used to call the appropriate settings, based on the run sensitivities function below

cases = {'baseline':           {'i_damage': 0,
                                'i_impeding_factors': 0,
                                'i_cordons': 0,
                                'description': 'Cordon Extent: 1.5H, REDi Impeding Factors'
                               },
         
        'tall_mitigation_contr':   {'i_damage': 0,
                                'i_impeding_factors': 1,
                                'i_cordons': 0,
                                'description': 'Contractors on contract'
                               },
         'tall_mitigation_eng':     {'i_damage': 0,
                                'i_impeding_factors': 2,
                                'i_cordons': 0,
                                'description': 'Engineers on contract'
                               },
         'tall_mitigation_finance': {'i_damage': 0,
                                  'i_impeding_factors': 3,
                                  'i_cordons': 0,
                                  'description': 'Buildings are insured'
                                 },
         'tall_mitigation_all': {'i_damage': 0,
                                 'i_impeding_factors': 4,
                                 'i_cordons': 0,
                                 'description': 'All preparedness plans are in place'
                                 },         
         'tall_retrofit':        {'i_damage': 1,
                                 'i_impeding_factors': 0,
                                 'i_cordons': 0,
                                 'description': 'Retrofitted to modern code requirements'
                                 },

         
         'cordon_extent':      {'i_damage': 0,
                                'i_impeding_factors': 0,
                                'i_cordons': 1,
                                'description': 'Cordon radius reduced to 1.0H'
                               },
         'no_cordon':          {'i_damage': 0,
                                'i_impeding_factors': 0,
                                'i_cordons': 2,
                                'description': 'No cordons (only impeding factors)'
                               },

        'all_bldgs_0.5_impeding_factors': {'i_damage': 0,
                                 'i_impeding_factors': 5,
                                 'i_cordons': 0,
                                 'description': 'Reduced by 50% for all buildings'
                                 },
         
         'cordons_no_impeding_factors': {'i_damage': 0,
                                 'i_impeding_factors': 0,
                                 'i_cordons': 3,
                                 'description': 'None prior to building stabilization repairs'
                                 },   
        }

#### Define the analysis cases in relation to the indices for damage, impeding factor, and cordon simulations

In [6]:
def run_sensitivities(input_filenames, output_filename, i_analysis):
    
    [bldgs, ruptures, gm_sites, gm_periods, ground_motion] = load_inputs(inventory_filename, ground_motion_filename)
    bldg_ids = bldgs.index.values
    
    [i_damage, i_impeding_factors, i_cordons] = i_analysis
    
    hgt = 75 *3.28  # tall building height cutoff in ft

    # create dictionary for sensitivity settings
    options = dict()
    
    # set time steps
    options['dt'] = 30

    # set number of realizations
    n_realizations = 1000
    options['n_realizations'] = n_realizations
    
    # occupancy labels for functions of interest
    occ_labels = ['Residential', 'Commercial Office', 'All Occupancies']
    options['occ_labels'] = occ_labels

    # set mitigation inputs
    bldgs = pd.read_csv(inventory_filename)
    bldgs = bldgs.set_index('id')

    mitigation = pd.DataFrame(index=bldgs.index)
    mitigation['facility'] = 'Non-Essential'
    mitigation['inspection'] = 'Default'
    mitigation['eng_mob'] = 'Default'
    mitigation['financing'] = 'Private Loans'
    mitigation['contr_mob'] = 'Default'
    mitigation['permitting'] = 'Default'

    if i_impeding_factors != 0:
        tall_idx = bldgs['building.building_ht_ft'] >= hgt
        if i_impeding_factors == 1:
            mitigation.loc[tall_idx, 'contr_mob'] = 'On Contract'
        elif i_impeding_factors == 2:
            mitigation.loc[tall_idx, 'eng_mob'] = 'On Contract'
        elif i_impeding_factors == 3:
            mitigation.loc[tall_idx, 'financing'] = 'Insurance'
        elif i_impeding_factors == 4:
#             mitigation.loc[tall_idx, 'inspection'] = 'BORP'
            mitigation.loc[tall_idx, 'eng_mob'] = 'On Contract'
            mitigation.loc[tall_idx, 'financing'] = 'Insurance'
            mitigation.loc[tall_idx, 'contr_mob'] = 'On Contract'

    options['mitigation'] = mitigation
    
    
    # set retrofit status
    retrofit_status = pd.DataFrame(index=bldgs.index)
    retrofit_status['status'] = 'original'
    
    if i_damage == 1:
        tall_idx = bldgs['building.building_ht_ft'] >= hgt
        retrofit_status.loc[tall_idx,'status'] = 'retrofitted'
    options['retrofit_status'] = retrofit_status

    # set impeding factor weights
    impeding_factor_building_weights = np.ones([len(bldgs),9])

    if i_impeding_factors == 5:
        impeding_factor_building_weights[:] = 0.5
    if i_impeding_factors == 6:
        impeding_factor_building_weights[np.where(tall_idx)[0],:] = 0

    options['impeding_factor_building_weights'] = impeding_factor_building_weights

    # can include community damage weights to scale for demand surge
    if i_impeding_factors == 7:
        options['community_damage_weights'] = {'community_damage_parameter': 11,
                                                'community_damage_threshold': 0.25,
                                                'min_community_damage': 0.2,
                                                'max_community_damage': 0.8,
                                                'min_damage_weight': 0.5,
                                                'max_damage_weight': 1.5}

    
    
    # set cordon rules
    options['height_threshold'] = hgt
    options['radius_scale_factor'] = 1.5
    options['cordon_duration_parameter'] = 'stable downtime'
        
    options['cordon_trigger_parameters'] = {'trigger_condition_parameter': ['building.year_of_construction','building.year_of_construction'],
                                            'trigger_condition_value': [2000,2000],
                                            'trigger_condition_logical': ['<', '>='],
                                            'cordon_damage_indicator': ['max_sdr', 'max_sdr'],
                                            'cordon_damage_threshold': [0.02, 0.04]}
    
    if i_cordons == 1:
        options['radius_scale_factor'] = 1.0
    if i_cordons == 2:
        options['radius_scale_factor'] = 0.0
    if i_cordons == 3:
        options['cordon_duration_parameter'] = 'stable repair'
        
    if options['cordon_duration_parameter'] == 'stable repair':
        options['replacement_stabilization_time'] = 30
        
#     print(options)


    # retrieve results
    [bldgs, community_damage, community_downtime] = load_results(input_filenames, output_filename, i_analysis, options)
    

#### Run the analysis. Any case indices that are already stored will be loaded, rather than rerun.

In [7]:
case_names = list(cases.keys())

for case in case_names:
    
    i_damage = cases[case]['i_damage']
    i_impeding_factors = cases[case]['i_impeding_factors']
    i_cordons = cases[case]['i_cordons']
    
    i_analysis = [i_damage, i_impeding_factors, i_cordons]
    
    print('Running case: ' + case + '.')
    print()
    run_sensitivities(input_filenames, output_filename, i_analysis)
    print('Case complete: ' + case + '.')
    print()
    print()
    
with h5py.File(output_filename, 'r+') as hf:
    hf['Results'].attrs['contents'] = str(cases)
print('All cases complete.')

Running case: baseline.

File created
Damage sampled
Impeding factors sampled
Cordons evaluated

Case complete: baseline.


Running case: tall_mitigation_contr.

Damage loaded
Impeding factors sampled
Cordons evaluated

Case complete: tall_mitigation_contr.


Running case: tall_mitigation_eng.

Damage loaded
Impeding factors sampled
Cordons evaluated

Case complete: tall_mitigation_eng.


Running case: tall_mitigation_finance.

Damage loaded
Impeding factors sampled
Cordons evaluated

Case complete: tall_mitigation_finance.


Running case: tall_mitigation_all.

Damage loaded
Impeding factors sampled
Cordons evaluated

Case complete: tall_mitigation_all.


Running case: tall_retrofit.

Damage sampled
Impeding factors sampled
Cordons evaluated

Case complete: tall_retrofit.


Running case: cordon_extent.

Damage loaded
Impeding factors loaded
Cordons evaluated

Case complete: cordon_extent.


Running case: no_cordon.

Damage loaded
Impeding factors loaded
Cordons evaluated

Case complete